In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

#to display all rows columns 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [58]:
df = pd.read_csv(r"C:\Users\rohit\Downloads\ipl_csv2\latest_ipl_data.csv")

In [59]:
# df.head()

In [60]:
df['total_runs'] = df['runs_off_bat'] + df['extras']

In [61]:
# df.head()

In [62]:
# df.innings.unique()

In [63]:
df = df[(df.innings == 1) | (df.innings == 2)]

In [64]:
df['player_dismissed'] = df['player_dismissed'].fillna(0)

In [65]:
# df.head()

In [66]:
df['wicket'] = np.where(df['player_dismissed'] == 0,0,1)

In [67]:
# df.head()

In [68]:
# df.batting_team.unique()

In [69]:
t1 = 'Gujarat Titans'
t2 = 'Chennai Super Kings'

In [70]:
t1_outcomes_count = df[df['batting_team'] == t1]['runs_off_bat'].value_counts()
t2_outcomes_count = df[df['batting_team'] == t2]['runs_off_bat'].value_counts()

In [71]:
# t1_outcomes_count

In [72]:
# t2_outcomes_count


In [73]:
t1_outs = df[df['batting_team']== t1]['wicket'].sum()
t2_outs = df[df['batting_team']== t2]['wicket'].sum()

In [74]:
# t1_outs

In [75]:
outcomes = [0,1,2,3,4,6,'w']

In [76]:
t1_outcomes = []
t2_outcomes = []


for outcome in outcomes:
    if outcome!= 'w':
        t1_outcomes.append(t1_outcomes_count[outcome])
        t2_outcomes.append(t2_outcomes_count[outcome])
    else:
        t1_outcomes.append(t1_outs)
        t2_outcomes.append(t2_outs)
        

In [77]:
# t1_outcomes

In [78]:
t1_prob_ls = pd.Series([i/sum(t1_outcomes) for i in  t1_outcomes])

In [79]:
# t1_prob_ls

In [80]:
t2_prob_ls = pd.Series([i/sum(t2_outcomes) for i in  t2_outcomes])

In [81]:
t1_ls = t1_prob_ls.cumsum()

In [82]:
# t1_ls

In [83]:
t2_ls = t2_prob_ls.cumsum()

In [84]:
# t2_ls

In [85]:
# df[(df['batting_team'] == t1) & (df['bowling_team'] == t2) & (df['innings'] == 1)].match_id.unique()

In [86]:
matchID = 1370353

In [87]:
mdf = df[df['match_id'] == matchID]
# mdf.tail()

In [88]:
df_ing1 = mdf[mdf['innings'] == 1]
df_ing2 = mdf[mdf['innings'] == 2]

In [89]:
df_ing1.reset_index(inplace=True,drop=True)
df_ing2.reset_index(inplace=True,drop=True)

In [90]:
# df_ing2.head()

In [91]:
# len(df_ing2)

In [92]:
over_ball = 19.5
over_no = int(str(over_ball).split('.')[0])
ball_no = int(str(over_ball).split('.')[1])

In [93]:
    i1p_0 = t1_ls[0]
    i1p_1 = t1_ls[1]
    i1p_2 = t1_ls[2]
    i1p_3 = t1_ls[3]
    i1p_4 = t1_ls[4]
    i1p_6 = t1_ls[5]
    i1p_w = t1_ls[6]



In [94]:
# i1p_w


In [95]:
def ing1_runs(curr_score,curr_wks,at_overs):
    pred_runs = curr_score
    pred_wks = curr_wks
    over_ball = at_overs
    over_no = int(str(over_ball).split('.')[0])
    ball_no = int(str(over_ball).split('.')[1])
    leftover_balls = len(df_ing1) - (over_no*6 + ball_no)
    #leftover_wks = 10-curr_wks

    i1p_0 = t1_ls[0]
    i1p_1 = t1_ls[1]
    i1p_2 = t1_ls[2]
    i1p_3 = t1_ls[3]
    i1p_4 = t1_ls[4]
    i1p_6 = t1_ls[5]
    i1p_w = t1_ls[6]

    for i in range(leftover_balls):
        r_value = np.random.random()
        if r_value <= i1p_0:
            pred_runs += 0
        elif r_value <= i1p_1:
            pred_runs += 1
        elif r_value <= i1p_2:
            pred_runs += 2
        elif r_value <= i1p_3:
            pred_runs += 3
        elif r_value <= i1p_4:
            pred_runs += 4
        elif r_value <= i1p_6:
            pred_runs += 6
        else:
            pred_runs += 0
            pred_wks += 1
            
            if pred_wks == 10:
                break
        #if pred_wks == 10:
                    #break       

    return(pred_runs)
            
                
                
                


In [96]:
# ing1_runs(50,2,5.1)

In [97]:
curr_score = 0
curr_wickets = 0
curr_overs = 0.0

ing1_runs_pred = []

for i in range(len(df_ing1)):
    curr_score += df_ing1.total_runs[i]
    curr_wickets += df_ing1.wicket[i]
    curr_overs = df_ing1.ball[i]
    
    prediction = ing1_runs(curr_score, curr_wickets, curr_overs)
    ing1_runs_pred.append(prediction)

In [98]:
#ing1_runs_pred

In [99]:
actual_score = df_ing1['total_runs'].sum()

In [100]:
# x = [i for i in range(len(df_ing1))]
# y = ing1_runs_pred
# plt.figure(figsize = (16,6))
# plt.scatter(x,y)
# plt.ylim(0,actual_score +30)
# plt.axhline(204,ls= '--')
# plt.text(0,204,f'actual score:{actual_score}')

In [101]:
df_ing2['cum_runs'] = df_ing2['total_runs'].cumsum()

In [102]:
# df_ing2.head()

In [103]:
# sum(df_ing1['total_runs'])

In [104]:
def ing2_runs(curr_score,curr_wks,at_overs,target):
    pred_runs = curr_score
    pred_wks = curr_wks
    over_ball = at_overs
    over_no = int(str(over_ball).split('.')[0])
    # ball_no = int(str(over_ball).split('.')[1])
    leftover_balls = len(df_ing2) - (over_no*6)
    #leftover_wks = 10-curr_wks

    i1p_0 = t1_ls[0]
    i1p_1 = t1_ls[1]
    i1p_2 = t1_ls[2]
    i1p_3 = t1_ls[3]
    i1p_4 = t1_ls[4]
    i1p_6 = t1_ls[5]
    i1p_w = t1_ls[6]

    for i in range(leftover_balls):
        r_value = np.random.random()
        if r_value <= i1p_0:
            pred_runs += 0
        elif r_value <= i1p_1:
            pred_runs += 1
        elif r_value <= i1p_2:
            pred_runs += 2
        elif r_value <= i1p_3:
            pred_runs += 3
        elif r_value <= i1p_4:
            pred_runs += 4
        elif r_value <= i1p_6:
            pred_runs += 6
        else:
            pred_runs += 0
            pred_wks += 1
            
            if pred_wks == 10:
                break
        if pred_runs > target:
            break       

    return(pred_runs)
            
                
                
                


In [105]:
# ing2_runs(87,1,6.1,204)

In [106]:
def win_game(pred_runs,target):
    if pred_runs > target:
        return 'win'
    elif pred_runs == target:
        return 'tie'
    else:
        return 'lose'

In [107]:
def find_runs(curr_score,curr_wks,at_overs,target):
    req_runs = []
    win_ls = []
    #pred_runs_ls = []
    #results_ls = []
    
    
    for i in range(curr_score,target+1):


        win_count = 0
        lose_count= 0
        tie_count =0
        
        for j in range(100):
            pred_runs = ing2_runs(i,curr_wks,at_overs,target)
            #pred_runs_ls.append(pred_runs)
            results_pred = win_game(pred_runs,target)
            #results_ls.append(results_pred)
            if results_pred == 'win':
                win_count += 1
            elif results_pred == 'tie':
                tie_count += 1
            else:
                 lose_count += 1
            win_ls.append(win_count)
            req_runs.append(i)
    required_runs = curr_score
    for i in range(len(req_runs)):
        if win_ls[i] >= 50:
            required_runs = req_runs[i]
            # print('Runs to be: ', req_runs[i])
            break
        if at_overs == 15:
            required_runs = target
            break
        if at_overs == 0:
             required_runs = curr_score + 1
             break
            #elif win == 'tie':
    return(required_runs)
    

In [108]:
# find_runs(0,1,0,167)

In [109]:
def find_wickets(curr_score,curr_wks,at_overs,target):

    req_wicket = []
    win_ls = []

    for i in range(curr_wks,10):
        win_count = 0
        tie_count = 0
        lose_count = 0

        for j in range(100):
            pred_runs = ing2_runs(curr_score,i,at_overs,target)
            #pred_runs_ls.append(pred_runs)
            results_pred = win_game(pred_runs,target)

            if results_pred == 'win':
                win_count += 1
            elif results_pred == 'tie':
                tie_count += 1
            else:
                lose_count += 1
        win_ls.append(win_count)
        req_wicket.append(i)

        required_wicket = curr_wks
        for i in range(len(req_wicket)):
            if win_ls[i] < 45:
                required_wicket = req_wicket[i]
                break
    return required_wicket       
    

In [110]:
# find_wickets(159,1,15.1,167)

In [111]:
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def find_runs_wickets(curr_score,curr_wks,at_overs,target):
   plt.figure(figsize = (16, 6))
    #x = np.linspace(-10, 10, num=1000)
   x = np.array(list(range(21)))
   req_value = find_runs(curr_score,curr_wks,at_overs,target)
   req_wk_value = find_wickets(curr_score,curr_wks,at_overs,target)

    
    # print(req_value)
    #y = np.array([req_value for i in range(21)])

    # plt.plot(x, current_overs * x + target_score)
    # plt.plot(x, y)
   plt.scatter(at_overs,req_value, s = 1200, color = 'red')
   plt.axhline(target,ls='--',c='b')
   plt.text(at_overs,req_value,f'{req_value}' + '/' + f'{req_wk_value}',ha='center',va = 'center')
   plt.text(at_overs,req_value-17,f'CSK needed to be {req_value}/{req_wk_value} after {at_overs} overs',ha='center',va = 'center')
   plt.text(3,target+10,f'CSK target :{target}',fontsize=15)
   plt.xticks(x)
   plt.ylim(0,target+50)
   plt.title('IPL Final 2023 : CSK VS GT' ,fontsize=20) 
   plt.xlabel('Overs',fontsize=15)
   plt.ylabel('Runs',fontsize=15) 
   plt.show() 
   


In [112]:
interactive_plot = interactive(find_runs_wickets,curr_score = widgets.IntSlider(min=1,max=250,step=1,value=0), curr_wks = widgets.IntSlider(min=0, max=10, step=1, value=1),  at_overs=widgets.IntSlider(min=3, max=15, step=1, value=10), target = widgets.IntSlider(min=0, max=250, step=1, value=130))
output = interactive_plot.children[-1]
output.layout.height = '500px'
interactive_plot

interactive(children=(IntSlider(value=1, description='curr_score', max=250, min=1), IntSlider(value=1, descrip…